# pandas: Filtering, grouping and sorting

Let's play around with features that can filter series, group data for aggregate functions and sort data. 

Through the following examples I was able to build the file `Melbourne_Annual.csv` from the data in the workbook `BOM_VIC_20210323.xlsx`.


In [26]:
import pandas as pd
import math
import datetime as dt

In [4]:
# Load data and prepare for use

bom = pd.read_excel('../data/BOM_VIC_20210323.xlsx')
bom = bom.drop(['Station'], axis=1)
bom = bom[:-82]

## Filtering

To create a filter in a `DataFrame`, you specify a "Boolean index" by creating an expression based on a `DataFrame`'s series.

This Boolean index is in reality a series of `True` and `False` that matches the expression and the index in the `DataFrame`.

You can create an expression using the typical equality operators `==`, `!=`, `<`, `>`, `<=`, `>=`. 

These expressions can be combined into more complex Boolean expressions by using AND `&`, OR `|` and NOT `~`.

In [5]:
# Find all dates when the temperature was greater than 45 Celsius
temp_filter = bom['Maximum'] >= 45
bom[temp_filter]

,Year,Month,Day,Rainfall,Maximum,Minimum
30692,1939,1,13,0.0,45.6,16.9
56277,2009,1,30,0.0,45.1,25.7
56285,2009,2,7,0.0,46.4,18.7


In [6]:
# Find all dates when there was more than 100mm of rain
rainfall_filter = bom['Rainfall'] > 100
bom[rainfall_filter]

,Year,Month,Day,Rainfall,Maximum,Minimum
39474,1963,1,29,108.0,21.2,16.1
54820,2005,2,3,113.4,19.5,11.0


In [7]:
# Find all dates when more than 10mm of rain fell and the maximum temperature was above 30 Celsius
temp_filter = bom['Maximum'] >= 30
rainfall_filter = bom['Rainfall'] >= 10
bom[temp_filter & rainfall_filter]

,Year,Month,Day,Rainfall,Maximum,Minimum
438,1856,3,14,10.7,31.0,20.3
1146,1858,2,20,34.5,34.7,11.7
2577,1862,1,21,10.9,32.3,13.9
3286,1863,12,31,19.1,30.0,16.8
4100,1866,3,24,13.2,30.0,16.3
5141,1869,1,28,15.5,36.7,16.7
6185,1871,12,8,16.5,31.7,15.5
7347,1875,2,12,19.6,34.1,15.8
8081,1877,2,15,13.7,31.0,17.3
12419,1889,1,1,26.4,31.1,20.0


In [8]:
# Find all dates when more than 10mm of rain fell or the maximum temperature was above 30 Celsius
bom[temp_filter | rainfall_filter]

,Year,Month,Day,Rainfall,Maximum,Minimum
114,1855,4,25,20.6,NaN,NaN
117,1855,4,28,15.2,NaN,NaN
118,1855,4,29,14.2,NaN,NaN
147,1855,5,28,12.2,12.7,7.2
162,1855,6,12,11.7,8.9,1.7
...,...,...,...,...,...,...
60612,2020,12,13,0.0,30.2,16.0
60613,2020,12,14,0.0,33.7,17.7
60614,2020,12,15,0.0,33.5,20.4
60621,2020,12,22,13.2,20.4,13.8


In [9]:
# What a Boolean index looks like
temp_filter.tail()

60626     True
60627    False
60628    False
60629    False
60630    False
Name: Maximum, dtype: bool

In [10]:
# You can also slice then filter and extract
bom.iloc[-366:][temp_filter][['Month','Day', 'Maximum']]

<ipython-input-10-ff0888617a66>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bom.iloc[-366:][temp_filter][['Month','Day', 'Maximum']]


,Month,Day,Maximum
60267,1,3,36.6
60273,1,9,31.9
60274,1,10,32.9
60277,1,13,32.2
60278,1,14,33.8
60279,1,15,36.8
60286,1,22,31.7
60293,1,29,31.5
60294,1,30,39.4
60295,1,31,42.9


In [16]:
# Let's create a histogram of maximum temperatures
# Define bins according to the integer part of a temperature value
max = bom['Maximum'][bom['Maximum'].notna()].apply(lambda t: math.floor(t))
histogram = max.value_counts()

# And sort the bins (which is the index for this Series)
# Now we should see how often a certain maximum is reached
histogram.sort_index()

4        1
5        1
6        6
7       31
8       85
9      237
10     748
11    1762
12    2932
13    3956
14    4217
15    4473
16    4503
17    4197
18    4133
19    3875
20    3652
21    3373
22    2853
23    2314
24    1816
25    1680
26    1508
27    1181
28    1100
29     865
30     878
31     725
32     689
33     585
34     490
35     447
36     352
37     267
38     215
39     139
40     115
41      52
42      28
43      20
44       5
45       2
46       1
Name: Maximum, dtype: int64

## Grouping

A `DataFrame` may be split into groups according to specific criteria. These groups can then have aggregate functions applied to them to produce a meaningful dataset.


In [70]:
# Simple grouping
df = bom.groupby(['Month', 'Day']).mean()

# See how `Month` and `Day` are the grouping indexes
df[['Maximum', 'Minimum', 'Rainfall']].head(15)


Maximum    Minimum  Rainfall
Month Day                                
1     1    24.774545  14.007273  1.062424
      2    25.373939  13.790909  2.217576
      3    25.591515  13.894545  2.174545
      4    25.165455  13.863030  1.623171
      5    25.249697  13.766667  1.859146
      6    24.783030  14.026667  1.736970
      7    25.480606  13.998788  1.509697
      8    26.320606  14.487273  1.352439
      9    26.341818  14.472727  1.488485
      10   26.456970  14.341212  1.286061
      11   26.042424  14.497576  1.101212
      12   25.761212  14.372727  1.106667
      13   26.427273  14.517576  1.487879
      14   26.404848  14.708485  1.615854
      15   26.243636  14.591515  1.822561

In [25]:
# Long-term mean monthly temperature
# As we are specifying the output series we need to specify the grouping objects as series
monthly_mean_temp_longterm = bom[['Minimum', 'Maximum']].groupby([bom['Month']]).mean()

# See how 'Month' is now an index in this `DataFrame`
print(monthly_mean_temp_longterm.index)

# Show
monthly_mean_temp_longterm

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64', name='Month')


,Minimum,Maximum
Month,,
1,14.411906,25.954467
2,14.696996,25.839562
3,13.309814,23.929717
4,10.857818,20.348434
5,8.725909,16.749378
6,6.943322,14.101988
7,6.086242,13.516459
8,6.762728,14.990808
9,8.050201,17.268614


In [23]:
# Long-term mean monthly rainfall totals
# First calculate the sum for every month, then determine the monthly mean
# You may want to keep 'Month' in the output `DataFrame` so it can be used in a plot
df = bom[['Year','Month','Rainfall']].groupby([bom['Year'], bom['Month']]).agg({'Year': 'first', 'Month': 'first', 'Rainfall': 'sum'})
monthly_mean_rainfall_total_longterm = df[['Month', 'Rainfall']].groupby([df['Month']]).agg({'Month':'first','Rainfall':'mean'})

# Show
monthly_mean_rainfall_total_longterm


,Month,Rainfall
Month,,
1,1,47.037952
2,2,47.040964
3,3,49.178313
4,4,57.427108
5,5,55.559639
6,6,49.065060
7,7,47.262048
8,8,50.080120
9,9,57.404819


In [28]:
# Let's see what the means are for the last 40 years
# We'll use built-in `datetime` objects to help create the range we need
FORTY_YEARS_AGO = pd.to_datetime(dt.datetime(1981, 1, 1))
recent = bom.copy()
recent['Date'] = pd.Series([pd.to_datetime(dt.datetime(r.Year, r.Month, r.Day)) for r in recent.itertuples()])
recent = recent[recent['Date'] >= FORTY_YEARS_AGO]

In [32]:
# Mean temperature over the last 40 years
monthly_mean_temp_recent = recent[['Minimum', 'Maximum']].groupby([recent['Month']]).mean()

monthly_mean_temp_recent

,Minimum,Maximum
Month,,
1,15.892097,26.500806
2,16.072124,26.562478
3,14.658710,24.496532
4,11.900417,21.023520
5,9.875403,17.546048
6,7.848123,14.813583
7,7.268306,14.289839
8,7.885806,15.618145
9,9.278250,17.804000


In [33]:
# Mean monthly total rainfall for the last 40 years
df = recent[['Year','Month','Rainfall']].groupby([recent['Year'], recent['Month']]).agg({'Year': 'first', 'Month': 'first', 'Rainfall': 'sum'})
monthly_mean_rainfall_total_recent = df[['Month', 'Rainfall']].groupby([df['Month']]).agg({'Month':'first','Rainfall':'mean'})

# Show
monthly_mean_rainfall_total_recent

,Month,Rainfall
Month,,
1,1,45.280
2,2,41.625
3,3,39.240
4,4,53.280
5,5,48.090
6,6,46.820
7,7,44.185
8,8,49.770
9,9,51.680


In [34]:
# Are recent mean temperatures higher than the long-term mean?
monthly_mean_temp_recent > monthly_mean_temp_longterm


,Minimum,Maximum
Month,,
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,True,True
7,True,True
8,True,True
9,True,True


In [36]:
# Are recent mean rainfall totals lower than the long-term mean?
monthly_mean_rainfall_total_recent['Rainfall'] < monthly_mean_rainfall_total_longterm['Rainfall']

Month
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11    False
12    False
Name: Rainfall, dtype: bool

In [40]:
# This and the next code cell helped create the file `Melbourne_Annual.csv`
# Yearly mean temperature
mean_temperature_annual_longterm = bom['Maximum'].groupby([bom['Year']]).mean()
mean_temperature_annual_recent = recent['Maximum'].groupby([recent['Year']]).mean()

print(mean_temperature_annual_longterm.tail())
print()
print(mean_temperature_annual_recent.tail())


Year
2016    20.325410
2017    20.875616
2018    20.722192
2019    20.756164
2020    19.856557
Name: Maximum, dtype: float64

Year
2016    20.325410
2017    20.875616
2018    20.722192
2019    20.756164
2020    19.856557
Name: Maximum, dtype: float64


In [43]:
# Yearly total rainfall
total_rainfall_annual_longterm = bom['Rainfall'].groupby([bom['Year']]).sum()
total_rainfall_annual_recent = recent['Rainfall'].groupby([recent['Year']]).sum()

print(total_rainfall_annual_longterm.tail())
print()
print(total_rainfall_annual_recent.tail())

Year
2016    599.6
2017    602.0
2018    514.4
2019    374.4
2020    786.8
Name: Rainfall, dtype: float64

Year
2016    599.6
2017    602.0
2018    514.4
2019    374.4
2020    786.8
Name: Rainfall, dtype: float64


In [85]:
# Given the warm weather over late March and early April, what were the highest recorded temperatures reached for each of these days?

march_filter = (bom['Month'] == 3) & (bom['Day'] >= 24)
april_filter = (bom['Month'] == 4) & (bom['Day'] <= 7)

df = bom[march_filter | april_filter]
data = df.groupby([df['Month'],df['Day']]).agg(Highest_Temp=('Maximum', 'max'))

data

Highest_Temp
Month Day              
3     24           36.1
      25           36.1
      26           34.2
      27           35.6
      28           34.4
      29           37.8
      30           35.9
      31           34.4
4     1            33.4
      2            33.5
      3            32.3
      4            33.9
      5            34.9
      6            34.4
      7            33.9

In [87]:
# But when did it happen?
# (Probably not the prettiest code)

# The index for this `DataFrame` is a `MultiIndex` where the values of the groups is stored as a tuple
index = data.index.to_flat_index()

year = []
month = []
day = []
temp = []

for (m, d) in index:
    t = data.at[(m, d), 'Highest_Temp']
    r = df[(df['Month'] == m) & (df['Day'] == d) & (df['Maximum'] == t)].tail(1)
    y = r['Year'].iloc[0]
    year.append(y)
    month.append(m)
    day.append(d)
    temp.append(t)

highest_temps = pd.DataFrame(
    data = {
        'Year': year,
        'Month': month,
        'Day': day,
        'Highest_Temp': temp
    }
)

highest_temps

,Year,Month,Day,Highest_Temp
0,1882,3,24,36.1
1,1884,3,25,36.1
2,1884,3,26,34.2
3,2013,3,27,35.6
4,1888,3,28,34.4
5,1940,3,29,37.8
6,1940,3,30,35.9
7,1936,3,31,34.4
8,2014,4,1,33.4
9,1954,4,2,33.5


## Sorting

It is possible to sort the values of a `Series` or, as shown above, sort the index of a `DataFrame` or `Series`.


In [49]:
# Find the hottest days in Melbourne and sort from the highest down
df.sort_values(by='Maximum', ascending=False).head(10)

,Year,Month,Day,Rainfall,Maximum,Minimum
56285,2009,2,7,0.0,46.4,18.7
30692,1939,1,13,0.0,45.6,16.9
56277,2009,1,30,0.0,45.1,25.7
30689,1939,1,10,0.0,44.7,15.7
56276,2009,1,29,0.0,44.3,25.7
19373,1908,1,17,0.0,44.2,20.4
54080,2003,1,25,0.0,44.1,17.3
2570,1862,1,14,0.8,44.0,20.9
1847,1860,1,22,0.0,43.9,18.0
58089,2014,1,16,0.0,43.9,27.0
